## This notebook partially relies on code+data from [1] that are not publicly available. 

[1] Aris Anagnostopoulos, Luca Becchetti, Adriano Fazzone, Cristina Menghini, and Chris Schwiegelshohn. 2020. Spectral Relaxations and Fair Densest Subgraphs. In Proceedings of the 29th ACM International Conference on Information & Knowledge Management (CIKM '20). Association for Computing Machinery, New York, NY, USA, 35–44. DOI:https://doi.org/10.1145/3340531.3412036

In [ ]:
import scipy.io as sio
import numpy as np
import networkx as nx
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import binarize
from algorithms import *
import dsd
import matplotlib
import random

import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams.update({'font.size': 16})

from matplotlib import pyplot as plt
plt.rcParams['text.usetex'] = True

In [ ]:
marketplaces = [f for f in listdir('../../CIKM20__Spectral_Relaxations_and_Fair_Densest_Subgraphs__sw/data/Amazon_metadata_2018') if ( isfile(join('../../CIKM20__Spectral_Relaxations_and_Fair_Densest_Subgraphs__sw/data/Amazon_metadata_2018', f)) and f.split('.')[1]=='tsv')]
datasets = dict()

In [ ]:
def read_amazon( uni_name ):
    prefix = '../../CIKM20__Spectral_Relaxations_and_Fair_Densest_Subgraphs__sw/data/Amazon_metadata_2018/'
    product_map = dict()
    prod = 0
    category_map = dict()
    cat = 0
    edges = []
    products_labels = dict()
    with open(prefix + uni_name + '.tsv','r') as f:
        first_line = f.readline()
        for line in f:
            line = line.rstrip()
            id1, id2, cat1, cat2 = line.split('\t')
            if id1 not in product_map:
                product_map[id1] = prod
                prod += 1
            if id2 not in product_map:
                product_map[id2] = prod
                prod += 1
            if cat1 not in category_map:
                category_map[cat1] = cat
                cat += 1
            if cat2 not in category_map:
                category_map[cat2] = cat
                cat += 1
            # Add edge
            edges.append([product_map[id1],product_map[id2]])
            # Create labels
            products_labels[product_map[id1]] = category_map[cat1]
            products_labels[product_map[id2]] = category_map[cat2]
    G = nx.Graph()
    G.add_edges_from( edges )
    labels = [products_labels[i] for i in range(G.number_of_nodes())]
    return nx.to_scipy_sparse_matrix(G),labels

In [ ]:
for uni in marketplaces:
    if True:
        uni_name = uni.split('.')[0]
        datasets[uni_name] = dict()
        graph,labels = read_amazon( uni_name )
        datasets[uni_name]['graph'] = graph
        datasets[uni_name]['labels'] = labels
        #datasets[uni_name]['G'] = nx.from_numpy_matrix(mat_contents['A'])
    #except:
    #    print("Error")
    #    pass

In [ ]:
ds_size = defaultdict( int )
ds_dist = defaultdict( list )
algo2_size = dict()
algo2_dist = dict()
ps_size = defaultdict( int )
ps_dist = defaultdict( list )
#fps_size = defaultdict( int )
fps_dist = defaultdict( list )
ds_connected = defaultdict( bool )
algo2_connected = dict()
ps_connected = defaultdict( bool )
fps_connected = defaultdict( bool )

ds_diameter = defaultdict( int )
algo2_diameter = dict()
ps_diameter = defaultdict( int )
fps_diameter = defaultdict( int )

ds_nodes = defaultdict( int )
algo2_nodes = dict()
ps_nodes = defaultdict( int )
fps_nodes = defaultdict( int )

In [ ]:
def getLabelsDist( S, l ):
    maxL = max(l) + 1
    dist = [0 for _ in range(maxL)]
    for n in S:
        dist[l[n]] += 1
    return dist

def diameter_estimation( G, S ):
    # G shoud be the subgraph -- not the graph!
    samples = max(int(0.1*len(S)),min(100,len(S)))
    src_list = random.sample( S, samples)
    diameter = 0
    for src in src_list:
        res = nx.single_source_shortest_path_length(G, src)
        max_val = max(res.values())
        diameter = max(diameter,max_val)
    return diameter

In [ ]:
# Get largest connected component
dtsts_cnt = 0
for uni_name in datasets:
    if uni_name not in fps_size:
        continue
    try:
        print("---", uni_name, "----")
        G = nx.from_numpy_matrix(datasets[uni_name]['graph'])
        Gcc = max(nx.connected_components(G), key=len)
        Gcc = sorted(list(Gcc))
        datasets[uni_name]['graph'] = datasets[uni_name]['graph'][np.ix_(list(Gcc),list(Gcc))]
        datasets[uni_name]['labels'] = [datasets[uni_name]['labels'][i] for i in Gcc]
        G = G.subgraph(Gcc)
        G = nx.convert_node_labels_to_integers(G)
        labels = list(datasets[uni_name]['labels'])
        if max(labels)+1 == 2:
            pass
        else:
            continue
        # DSD
        edges = [e for e in G.edges()]
        flowless_result = dsd.flowless(edges, 4)
        DS, DS_density = flowless_result[0], flowless_result[1]
        ds_size[uni_name] = DS_density
        ds_dist[uni_name] = getLabelsDist( DS, labels )
        print("Densest Subgraph: {:.2f}".format(DS_density))

        ds_nodes[uni_name] = len(DS)
        # Connectivity & Diameter
        G_DS = G.subgraph(DS)
        ds_connected[uni_name] = nx.is_connected( G_DS )
        if ds_connected[uni_name]:
            ds_diameter[uni_name] = diameter_estimation( G_DS, DS )

    #     # dalkDS and dalkFairSubgraph
        algo2_size[uni_name] = dict()
        algo2_dist[uni_name] = dict()
        algo2_connected[uni_name] = dict()
        algo2_diameter[uni_name] = dict()
        algo2_nodes[uni_name] = dict()
        for a in [0.5,0.55,0.6,0.7,0.8,0.9,0.99]:
            print("Results for a = {}".format(a))
            try:
                FP, dalk_density = Algorithm2( G, labels, a )
                fairDS, colors = FP
                t = len(colors)
                fairDensity = G.subgraph(fairDS).size() / len(fairDS)
                print("Densest at least-k: {:.2f}".format(dalk_density))
                print("Fair at most alpha density: {:.2f}".format(fairDensity))
                print("Maximum group contribution: {:.2f}".format(max([len(v) for k,v in colors.items()])/len(fairDS)))
                print("Minimum group contribution: {:.2f}".format(min([len(v) for k,v in colors.items()])/len(fairDS)))
                size = fairDensity
                dist = getLabelsDist( fairDS,labels )

    #             # Connectivity & Diameter
                algo2_nodes[uni_name][a] = len(fairDS)
                G_DS = G.subgraph(list(fairDS))
                algo2_connected[uni_name][a] = nx.is_connected( G_DS )
                if algo2_connected[uni_name][a]:
                    algo2_diameter[uni_name][a] = diameter_estimation( G_DS, fairDS )
            except:
                print("Bug or not feasible solution under current setting.")
                size = 0
                dist = [0 for _ in range(max(labels)+1)]
            algo2_size[uni_name][a] = size
            algo2_dist[uni_name][a] = dist
        # Paired Sweep from Anagnostopoulos
        ps_ds, ps_size[uni_name] = pairedSweep(datasets[uni_name]['graph'], G, labels, fair=False )
        # Connectivity & Diameter
        ps_nodes[uni_name] = len(ps_ds)
        G_DS = G.subgraph(list(ps_ds))
        ps_connected[uni_name] = nx.is_connected( G_DS )
        if ps_connected[uni_name]:
            ps_diameter[uni_name] = diameter_estimation( G_DS, ps_ds )

        fps_ds, fps_size[uni_name] = pairedSweep(datasets[uni_name]['graph'], G, labels, fair=True )
        # Connectivity & Diameter
        fps_nodes[uni_name] = len(fps_ds)
        G_DS = G.subgraph(list(fps_ds))
        fps_connected[uni_name] = nx.is_connected( G_DS )
        if fps_connected[uni_name]:
            fps_diameter[uni_name] = diameter_estimation( G_DS, fps_ds )

        print("PS and FPS ", ps_size[uni_name],fps_size[uni_name])
        print("***********")
        dtsts_cnt += 1
        print("Datasets count ",dtsts_cnt)
    except:
        pass

In [ ]:
# Compare algorithms and plot
cnt = 0
for uni_name in datasets:
    dataset = uni_name
    # DS
    ds = ds_size[dataset]
    max_a = max(ds_dist[dataset])/sum(ds_dist[dataset])
    print(max_a)
    plt.plot(max_a,ds,label='DS', color='black',marker='x',markersize=12)

    x,y = [],[]
    for a,val in algo2_size[dataset].items():
        if val > 0:
            next_x = max(algo2_dist[dataset][a])/sum(algo2_dist[dataset][a])
            x.append(next_x)
            y.append(val)
        else:
            x.append(a)
            y.append(val)
    plt.plot(x,y,label='Algorithm 2',linestyle='--',marker='o')
    plt.plot([0.25], ps_size[dataset],marker='s',label='Paired Sweep')
    plt.plot([0.25], fps_size[dataset],marker='+',label='Fair Paired Sweep')
    plt.legend()
    plt.title(dataset)
    plt.xlabel(r'Color Participation Upper Bound ($a$)')
    plt.ylabel('Density')
    #plt.savefig("plots/individual/"+dataset+".eps", format='eps', bbox_inches ="tight")
    plt.show()
    cnt += 1
    if cnt > 10:
        break

In [ ]:
# Plot general statistics
DenseSubgraph = [val for key,val in ds_size.items()]
ds_a = []
for dataset in ds_dist:
    max_a = max(ds_dist[dataset])/sum(ds_dist[dataset])
    ds_a.append( max_a )
#print(np.mean(ds_a))
plt.errorbar(np.mean(ds_a),np.mean(DenseSubgraph),xerr=np.std(ds_a),marker='x')
print("1. Dense Subgraph\t Mean:{:.2f}, Std:{:.2f}".format(np.mean(DenseSubgraph),np.std(DenseSubgraph)))
print("2. Algorithm 2")
x,y,err = [],[],[]
for a in [0.5,0.55,0.6,0.7,0.8,0.9,0.99]:
    DS = [key_val[a] for key_val in list(algo2_size.values())]
    print("alpha = {}\tMean: {:.2f}, Std: {:.2f}".format(a,np.mean(DS),np.std(DS)))
    x.append(a)
    y.append(np.mean(DS))
    err.append(np.std(DS))
plt.errorbar(x,y,yerr=err)
plt.title('All Datasets')
plt.xlabel('Color Participation Upper Bound (a)')
plt.ylabel('Density')
#plt.savefig("all_data.png", bbox_inches ="tight")
PS = [val for key,val in ps_size.items()]
FPS = [val for key,val in fps_size.items()]
print("3. Paired Sweep\t Mean:{:.2f}, Std:{:.2f}".format(np.mean(PS),np.std(PS)))
print("4. (Fair) Paired Sweep\t Mean:{:.2f}, Std:{:.2f}".format(np.mean(FPS),np.std(FPS)))


print(" ---- Nodes -----")
ds_mean = np.mean(list(ds_nodes.values()))
ds_std = np.std(list(ds_nodes.values()))
print("1. Dense Subgraph\t Mean:{:.2f}, Std:{:.2f}".format(ds_mean,ds_std))
for a in [0.5,0.55,0.6,0.7,0.8,0.9,0.99]:
    a_mean = np.mean([key_val[a] for key_val in list(algo2_nodes.values())])
    a_std = np.std([key_val[a] for key_val in list(algo2_nodes.values())])
    print("alpha = {}\tMean: {:.2f}, Std: {:.2f}".format(a,a_mean,a_std))
ps_mean = np.mean(list(ps_nodes.values()))
ps_std = np.std(list(ps_nodes.values()))

fps_mean = np.mean(list(fps_nodes.values()))
fps_std = np.std(list(fps_nodes.values()))

print("3. Paired Sweep\t Mean:{:.2f}, Std:{:.2f}".format(ps_mean,ps_std))
print("4. (Fair) Paired Sweep\t Mean:{:.2f}, Std:{:.2f}".format(fps_mean,fps_std))


print(" ---- Connected ----- ")
ds_mean = np.sum(list(ds_connected.values()))
print("1. Dense Subgraph\t:{:.2f}".format(ds_mean))
for a in [0.5,0.55,0.6,0.7,0.8,0.9,0.99]:
    a_mean = np.sum([key_val[a] for key_val in list(algo2_connected.values())])
    print("alpha = {}\t: {:.2f}".format(a,a_mean))
ps_mean = np.sum(list(ps_connected.values()))

fps_mean = np.sum(list(fps_connected.values()))

print("3. Paired Sweep\t Mean:{:.2f}".format(ps_mean,ps_std))
print("4. (Fair) Paired Sweep\t Mean:{:.2f}".format(fps_mean))

print(" ---- Diameter -----")
ds_g = set([v for v in ds_connected if ds_connected[v]])
a_g = set([v for v in algo2_connected if algo2_connected[v][0.5]])
f_g = set([v for v in fps_connected if fps_connected[v]])
temp = ds_g.intersection(a_g)
temp = temp.intersection(f_g)

ds_mean = np.mean([ds_diameter[g] for g in temp])
ds_std = np.std([ds_diameter[g] for g in temp])
print("1. Dense Subgraph\t Mean:{:.2f}, Std:{:.2f}".format(ds_mean,ds_std))
a_mean = np.mean([algo2_diameter[g][0.5] for g in temp])
a_std = np.std([algo2_diameter[g][0.5] for g in temp])
print("2. alpha = {}\tMean: {:.2f}, Std: {:.2f}".format(0.5,a_mean,a_std))
fps_mean = np.mean([fps_diameter[g] for g in temp])
fps_std = np.std([fps_diameter[g] for g in temp])
print("3. (Fair) Paired Sweep\t Mean:{:.2f}, Std:{:.2f}".format(fps_mean,fps_std))

In [ ]:
# Plot fairness distribution
DS_results = defaultdict(list)
for dataset in ds_dist:
    f_dist = sorted(ds_dist[dataset])
    sum_f_dist = sum(f_dist)
    f_dist = [x/sum_f_dist for x in f_dist]
    DS_results['0'].append(f_dist[0])
    DS_results['1'].append(f_dist[1])
fig, ax = plt.subplots()
ax.boxplot(DS_results.values())
ax.set_xticklabels(DS_results.keys())
plt.title('Densest Subgraph')
plt.ylim(0,1)
plt.savefig("dense_subgraph_amazon_boxplot.eps", format="eps", bbox_inches ="tight")
plt.show()

# Do it with Algo2
for a in [0.5,0.55,0.6,0.7,0.8,0.9,0.99]:
    temp_results = defaultdict(list)
    for dataset in algo2_dist:
        f_dist = sorted(algo2_dist[dataset][a])
        sum_f_dist = sum(f_dist)
        if sum_f_dist == 0:
            continue
        f_dist = [x/sum_f_dist for x in f_dist]
        temp_results['0'].append(f_dist[0])
        temp_results['1'].append(f_dist[1])
    fig, ax = plt.subplots()
    ax.boxplot(temp_results.values())
    ax.set_xticklabels(temp_results.keys())
    plt.title(r'$\alpha = {}$'.format(a))
    plt.ylim(0,1)
    plt.xlabel("Color (ordered by size)")
    plt.ylabel('Fraction of color in DS')
    plt.savefig(str(a)+"_boxplot_amazon.eps", format="eps", bbox_inches ="tight")
    plt.show()
        
    

In [ ]:
# Plot averages for all colors
# Plot DS
x_ds = np.mean([max(v)/sum(v) for v in list(ds_dist.values())])
y_ds = np.mean([v for v in list(ds_size.values())])
plt.plot(x_ds,y_ds,marker='x',markersize=18,label="DSD",color='black')


# Plot FPS
x_fps = 0.5
y_fps = np.mean([v for v in list(fps_size.values())])
plt.plot(x_fps,y_fps,marker='s',markersize=18,label="Fair PS",color='red')

# Plot PS
x_ps = 0.5
y_ps = np.mean([v for v in list(ps_size.values())])
plt.plot(x_ps,y_ps,marker='^',markersize=18,label="PS",color='green')

# Plot Algo2
x_a = []
y_a = []
for a in [0.5,0.55,0.6,0.7,0.8,0.9,0.99]:
    y_a.append( np.mean([v[a] for _,v in algo2_size.items()]) )
    x_a.append( np.mean([max(v[a])/sum(v[a]) for _,v in algo2_dist.items()]) )
plt.plot(x_a,y_a,marker='o',label='Algo. 2',color='blue')

plt.ylabel('Density')
plt.xlabel(r'Color Participation Upper Bound ($\alpha$)')
plt.legend()
plt.title('Amazon Products')
plt.savefig('amazon_total.eps',format='eps', bbox_inches ="tight")
plt.show()

# Plot DS
x_ds = np.mean([max(v)/sum(v) for v in list(ds_dist.values())])
y_ds = np.mean([v for v in list(ds_size.values())])
plt.plot(x_ds,y_ds/y_ds,marker='x',markersize=18,label="DSP",color='black')


# Plot FPS
x_fps = 0.5
y_fps = np.mean([v/y_ds for v in list(fps_size.values())])
plt.plot(x_fps,y_fps,marker='s',markersize=18,label="Fair PS",color='red')

# Plot PS
x_ps = 0.5
y_ps = np.mean([v/y_ds for v in list(ps_size.values())])
plt.plot(x_ps,y_ps,marker='^',markersize=18,label="PS",color='green')

# Plot Algorithm2
x_a = []
y_a = []
for a in [0.5,0.55,0.6,0.7,0.8,0.9,0.99]:
    y_a.append( np.mean([v[a]/y_ds for _,v in algo2_size.items()]) )
    x_a.append( np.mean([max(v[a])/sum(v[a]) for _,v in algo2_dist.items()]) )
plt.plot(x_a,y_a,marker='o',label='Algo. 2',color='blue')

plt.ylabel('Normalized Density')
plt.xlabel(r'Color Participation Upper Bound ($\alpha$)')
plt.ylim(0,1.05)
plt.legend(loc='lower left')
plt.title('Amazon Products')
plt.savefig('amazon_total_normaled.eps',format='eps', bbox_inches ="tight")
plt.show()


